In [1]:
# Import PyAgrum package 
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb

#### Let us initiate the process of creating an influence diagram, initially the diagram is empty.

In [2]:
diag=gum.InfluenceDiagram()
print(diag)

Influence Diagram{
  chance: 0,
  utility: 0,
  decision: 0,
  arcs: 0,
  domainSize: 1
}


#### We are about to purchase a car and factors taken into consideration are the Traffic(T)(High/Low), Mileage(M) (Low,Medium,High) , Category(C) (Normal,SUV) ,Performance(P) (Bad,Good) and Review(Bad,Good), for making decision whether to buy(Yes,No). 

#### The variables that our model depends on are called as chanceNodes in pyAgrum and let us create those.

In [3]:
## Creating all chance nodes.
t=diag.addChanceNode(gum.LabelizedVariable('T','Traffic',2))
m=diag.addChanceNode(gum.LabelizedVariable('M','Mileage',3))

c=diag.addChanceNode(gum.LabelizedVariable('C','category',2))
p=diag.addChanceNode(gum.LabelizedVariable('P','performance',2))

r=diag.addChanceNode(gum.LabelizedVariable("R","review",2))

In [4]:
#Let us now add our decision node.
b=diag.addDecisionNode(gum.LabelizedVariable('B','buy',2))

#### Once we make a decision, let us assume we have a way to model the satisfaction we get out of that decision.

In [5]:
# creating utility node.
s=diag.addUtilityNode(gum.LabelizedVariable("S","Satisfaction",1))

#### Having created the node , now we need to define the relationships between them.

In [6]:
diag.addArc(diag.idFromName('T'),diag.idFromName('M'))
diag.addArc(diag.idFromName('C'),diag.idFromName('M'))
diag.addArc(diag.idFromName('M'),diag.idFromName('R'))
diag.addArc(diag.idFromName('C'),diag.idFromName('P'))
diag.addArc(diag.idFromName("R"),diag.idFromName("B"))
diag.addArc(diag.idFromName("B"),s)
diag.addArc(diag.idFromName("P"),s)

#### We can visualise the relationship

In [7]:
gnb.showInfluenceDiagram(diag,size="6",format="svg") 

#### Now we need to define the probability distribution associated with each of the nodes.
#### Traffic and Category are independent variables, hence they can be filled directly as below.

In [8]:
diag.cpt(t).fillWith([0.6,0.4])

(gum::Potential<double>@0x1ed2560) <T:0> :: 0.6 /<T:1> :: 0.4

In [9]:
diag.cpt(c).fillWith([0.7,0.3])

(gum::Potential<double>@0x209a8a0) <C:0> :: 0.7 /<C:1> :: 0.3

In [10]:
 diag.cpt(m).var_names

['C', 'T', 'M']

#### Mileage is dependent on category and traffic and hence we need to define the conditional probability distribution for each of these possibilities. Below the first line corresponds to the state where category takes the state 0 and traffic takes the state 0 and so on.

In [11]:
diag.cpt(m)[0,0,:] = [0.2,0.6,0.2]
diag.cpt(m)[0,1,:] = [0.05,0.25,0.7]
diag.cpt(m)[1,0,:] = [0.9,0.08,0.02]
diag.cpt(m)[ 1, 1,:] = [0.3,0.5,0.2]
diag.cpt(m)

(gum::Potential<double>@0x216afe0) <M:0|T:0|C:0> :: 0.2 /<M:1|T:0|C:0> :: 0.6 /<M:2|T:0|C:0> :: 0.2 /<M:0|T:1|C:0> :: 0.05 /<M:1|T:1|C:0> :: 0.25 /<M:2|T:1|C:0> :: 0.7 /<M:0|T:0|C:1> :: 0.9 /<M:1|T:0|C:1> :: 0.08 /<M:2|T:0|C:1> :: 0.02 /<M:0|T:1|C:1> :: 0.3 /<M:1|T:1|C:1> :: 0.5 /<M:2|T:1|C:1> :: 0.2

#### Similarly we define the conditional probability distribution for review.

In [12]:
diag.cpt(r)[0,:]=[0.9,0.1]
diag.cpt(r)[1,:]=[0.4,0.6]
diag.cpt(r)[2,:]=[0.99,0.01]
diag.cpt(r)

(gum::Potential<double>@0x2042e50) <R:0|M:0> :: 0.9 /<R:1|M:0> :: 0.1 /<R:0|M:1> :: 0.4 /<R:1|M:1> :: 0.6 /<R:0|M:2> :: 0.99 /<R:1|M:2> :: 0.01

In [13]:
# Conditional distribution for performance.
diag.cpt(p)[0,:]=[0.95,0.05]
diag.cpt(p)[1,:]=[0.2,0.8]
diag.cpt(p)

(gum::Potential<double>@0x1f4a560) <P:0|C:0> :: 0.95 /<P:1|C:0> :: 0.05 /<P:0|C:1> :: 0.2 /<P:1|C:1> :: 0.8

#### Our utility function is also defined similar to conditional probability tables, but instead of a probability distribution, utility is a scalar value.

In [14]:
diag.utility(s).var_names

['P', 'B', 'S']

In [15]:
diag.utility(s)[0,0,:]=3
diag.utility(s)[0,1,:]= 0
diag.utility(s)[1,0,:]=2
diag.utility(s)[1,1,:]=4
diag.utility(s)

(gum::Potential<double>@0x20b9710) <S:0|B:0|P:0> :: 3 /<S:0|B:1|P:0> :: 0 /<S:0|B:0|P:1> :: 2 /<S:0|B:1|P:1> :: 4

#### Having defined the model now we are ready to make the inference.

In [16]:
ie=gum.InfluenceDiagramInference(diag)

In [17]:
ie.makeInference()

In [18]:
print("Best decision for B : {0}".format(ie.getBestDecisionChoice(diag.idFromName("B"))))

print("For a maxEU : {0}".format(ie.getMEU()))

Best decision for B : 0
For a maxEU : 2.7249999999999996


#### It's also possible to fix some of the variables at a predefined state, lets try setting the performance at both possible states and observe what might be the best thing to do in each of those cases.

In [19]:
ie.eraseAllEvidence()
ie.setEvidence({diag.idFromName('P'):1}) # setting the performance to high.

In [20]:
ie.makeInference()
print("Best decision for B : {0}".format(ie.getBestDecisionChoice(diag.idFromName("B"))))

print("For a maxExpected Utility : {0}".format(ie.getMEU()))

Best decision for B : 1
For a maxExpected Utility : 1.1


In [21]:
ie.eraseAllEvidence()
ie.setEvidence({diag.idFromName('P'):0}) # setting the performance to low.

In [22]:
ie.makeInference()
print("Best decision for B : {0}".format(ie.getBestDecisionChoice(diag.idFromName("B"))))

print("For a maxExpected Utility : {0}".format(ie.getMEU()))

Best decision for B : 0
For a maxExpected Utility : 2.175


#### Thus we see its best to buy car when the performance is good(1) and best not to buy the car when the performance is low(0).

#### We can also predefine both performance and mileage and let's see its impact below.

In [23]:
ie.eraseAllEvidence()
ie.setEvidence({diag.idFromName('P'):0,diag.idFromName('M'):0})

In [24]:
ie.makeInference()
print("Best decision for B : {0}".format(ie.getBestDecisionChoice(diag.idFromName("B"))))

print("For a maxExpected Utility : {0}".format(ie.getMEU()))

Best decision for B : 0
For a maxExpected Utility : 0.3981


In [25]:
ie.eraseAllEvidence()
ie.setEvidence({diag.idFromName('P'):1,diag.idFromName('M'):2})

In [26]:
ie.makeInference()
print("Best decision for B : {0}".format(ie.getBestDecisionChoice(diag.idFromName("B"))))

print("For a maxExpected Utility : {0}".format(ie.getMEU()))

Best decision for B : 1
For a maxExpected Utility : 0.14432


#### In this notebook we learnt how to construct influence diagram and make decisions.